In [8]:
import numpy as np 
import pandas as pd 

In [2]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [10]:
data=pd.read_csv('dataset\data')
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\navee\AppData\Local\Temp\ipykernel_4216\3346886667.py:1: SyntaxWarning: invalid escape sequence '\d'
  data=pd.read_csv('dataset\data')


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [11]:
data.isnull().sum()

Category    0
Resume      0
dtype: int64

In [20]:
data.shape

(962, 2)

In [23]:
dp=data.sample(n=962,random_state=42)

In [24]:
dp.head()

,Category,Resume
344,Java Developer,TECHNICAL SKILLS Programming Languages: Java (...
381,Java Developer,"TECHNICALSKILLS SpringMVC, Hibernate, JDBC, Ja..."
351,Java Developer,Education Details \r\nJanuary 2013 Master of E...
778,Hadoop,"Skill Set: Hadoop, Map Reduce, HDFS, Hive, Sqo..."
277,Health and fitness,Education Details \r\nJanuary 1992 to January ...


In [26]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
##remove speacial charectors
dp['Resume']=dp['Resume'].apply(lambda x:re.sub('[^a-z A-Z 0-9]','',x))
##remove stopwords
dp['Resume']=dp['Resume'].apply(lambda x:" ".join([ wor for wor in x.split() if wor not in stopwords.words('english')]))
##remove html tags
dp['Resume']=dp['Resume'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())
#remove links
dp['Resume']=dp['Resume'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))
##remove spaces
dp['Resume']=dp['Resume'].apply(lambda x:" ".join(x.split()))

In [29]:
def convert_word(word):
    return " ".join(WordNetLemmatizer().lemmatize(i) for i in word.split())
dp['Resume']=dp['Resume'].apply(lambda x:convert_word(x))
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dp['Resume'],dp['Category'],test_size=0.2,random_state=42)
from sklearn.feature_extraction.text import CountVectorizer
con=CountVectorizer()
x_train_con=con.fit_transform(x_train)
x_test_con=con.transform(x_test)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_con=LogisticRegression()
model_con.fit(x_train_con,y_train)
y_predict_con=model_con.predict(x_test_con)
accu=accuracy_score(y_test,y_predict_con)
print(accu)
print(classification_report(y_test,y_predict_con))

0.9948186528497409
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         7
                     Arts       1.00      1.00      1.00        11
       Automation Testing       1.00      1.00      1.00         6
               Blockchain       0.83      1.00      0.91         5
         Business Analyst       1.00      1.00      1.00        10
           Civil Engineer       1.00      1.00      1.00         6
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.83      0.91         6
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         9
   Electrical Engineering       1.00      1.00      1.00         5
                       HR       1.00      1.00      1.00        12
                   Hadoop       1.00      

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
x_train_tf=tf.fit_transform(x_train)
x_test_tf=tf.transform(x_test)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_tf=LogisticRegression()
model_tf.fit(x_train_tf,y_train)
y_predict_tf=model_tf.predict(x_test_tf)
accu=accuracy_score(y_test,y_predict_tf)
print(accu)
print(classification_report(y_test,y_predict_tf))

0.9948186528497409
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         7
                     Arts       1.00      1.00      1.00        11
       Automation Testing       1.00      1.00      1.00         6
               Blockchain       1.00      1.00      1.00         5
         Business Analyst       1.00      1.00      1.00        10
           Civil Engineer       1.00      1.00      1.00         6
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.83      0.91         6
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         9
   Electrical Engineering       1.00      1.00      1.00         5
                       HR       1.00      1.00      1.00        12
                   Hadoop       1.00      

In [31]:
from gensim.models import Word2Vec
w2vec_model=Word2Vec(sentences=x_train,vector_size=100,window=5)
def get_wordvec(tokens,model,vector_size=100):
    vectors=[model.wv[token] for token in tokens if token in model.wv]
    if len(vectors)==0:
        return np.zeros(vector_size)
    return np.mean(vectors,axis=0)
x_train_wo=np.array([get_wordvec(tokens,w2vec_model) for tokens in x_train])
x_test_wo=np.array([get_wordvec(tokens,w2vec_model) for tokens in x_test])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
model_w2=LogisticRegression()
model_w2.fit(x_train_wo,y_train)
y_predict_w2=model_w2.predict(x_test_wo)
accu=accuracy_score(y_test,y_predict_w2)
print(accu)
print(classification_report(y_test,y_predict_w2))

0.24870466321243523
                           precision    recall  f1-score   support

                 Advocate       0.00      0.00      0.00         7
                     Arts       0.67      0.36      0.47        11
       Automation Testing       0.00      0.00      0.00         6
               Blockchain       0.00      0.00      0.00         5
         Business Analyst       0.00      0.00      0.00        10
           Civil Engineer       0.00      0.00      0.00         6
             Data Science       0.00      0.00      0.00         5
                 Database       0.00      0.00      0.00         8
          DevOps Engineer       0.09      0.67      0.16         6
         DotNet Developer       0.00      0.00      0.00         5
            ETL Developer       0.00      0.00      0.00         9
   Electrical Engineering       0.00      0.00      0.00         5
                       HR       1.00      0.33      0.50        12
                   Hadoop       0.50     

c:\Users\navee\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\navee\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\navee\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
import pickle
with open('vector.pkl','wb') as f:
    vector=pickle.dump(tf,f)
with open('model.pkl','wb') as f:
    vector=pickle.dump(model_tf,f)